the dataset contains 25000 positive and 25000 negative reviews. The dataset was from the reviews given by users on imdb. It contains at most 300 reviews per movie

At least 7 stars out of 10 = positive(label =1)
At most 4 stars out of 10 = negative(label=0)

In [2]:
from google.colab import files
files.upload()

Saving movie_data.csv to movie_data.csv


In [10]:
import pandas as pd

In [11]:
df= pd.read_csv("movie_data.csv",engine='python',error_bad_lines=False)

I used the python engine instead of python since there was an error( Error tokenizing data. C error: EOF inside string starting at row 30033) atline 30033. 
I have also used error_bad_lines = False to skip the last line since it was throwing the error (unexpected end of data)

In [12]:
df.head(10)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0
3,hi for all the people who have seen this wonde...,1
4,"I recently bought the DVD, forgetting just how...",0
5,Leave it to Braik to put on a good show. Final...,1
6,Nathan Detroit (Frank Sinatra) is the manager ...,1
7,"To understand ""Crash Course"" in the right cont...",1
8,I've been impressed with Chavez's stance again...,1
9,This movie is directed by Renny Harlin the fin...,1


In [13]:
df['review'][0]

'In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70\'s, they discover the criminal and a net of power and money to cover the murder.<br /><br />"Murder in Greenwich" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich famil

<b>Features: bag of 1-grams with TF-IDF values</b>:
- Extremely sparse feature matrix - close to 97% are zeros

<b>Model: Logistic regression</b>
- $p(y = 1|x) = \sigma(w^{T}x)$
- Linear classification model
- Can handle sparse data
- Fast to train
- Weights can be interpreted


converting the dataset to tf-idf values

word relevancy using term frequency-inverse document frequency

$$\text{tf-idf}(t,d)=\text{tf (t,d)}\times \text{idf}(t,d)$$

$$\text{idf}(t,d) = \text{log}\frac{n_d}{1+\text{df}(d, t)},$$

where $n_d$ is the total number of documents, and df(d, t) is the number of documents d that contain the term t.

In [14]:
df.loc[0,'review'][-50:]

'is seven.<br /><br />Title (Brazil): Not Available'

html tags, breaks and commas are seen and we need to clean them out and replacing them with an empty string.<br/>
We are also looking for emojis and moving them towards the end

In [16]:
import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    return text

In [17]:
preprocessor(df.loc[0,'review'][-50:])

'is seven title brazil not available'

you can now notice that the punction is not included and that the html tags have also been stripped.

using apply from pandas we can now do this to the entire dataset

In [18]:
df['review'] = df['review'].apply(preprocessor)

<h3>Tokenization of documents</h3>

we'll use nltk

In [19]:
from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()

function for tokenization that takes text as an argument and returns tokens

In [20]:
def tokenizer(text):
  return text.split()

stem form of tokenized text

In [21]:
def tokenizer_porter(text):
  return [porter.stem(word) for word in text.split()]

testing if the functions work

In [24]:
tokenizer('Zeus is stronger than ares at fighting')

['Zeus', 'is', 'stronger', 'than', 'ares', 'at', 'fighting']

In [25]:
tokenizer_porter('Zeus is stronger than ares at fighting')

['zeu', 'is', 'stronger', 'than', 'are', 'at', 'fight']

<h3>transforming the dataset into tf-idf vectors</h3>

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer


tfidf = TfidfVectorizer(strip_accents=None, lowercase=False, preprocessor=None, tokenizer=tokenizer_porter,use_idf=True,norm='l2',smooth_idf=True)

y = df.sentiment.values
x = tfidf.fit_transform(df.review)


sentiment classification using logistic regression

let's train the model first

In [42]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, random_state=1, test_size=0.5,shuffle=False)

pickle for saving the model

In [43]:
import pickle
from sklearn.linear_model import LogisticRegressionCV

clf = LogisticRegressionCV(cv=5,
                           scoring='accuracy',
                           random_state=0,
                           n_jobs=-1,
                           verbose=3,
                           max_iter=300).fit(X_train, y_train)

saved_model = open('saved_model.sav', 'wb')
pickle.dump(clf, saved_model)
saved_model.close()
                          

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  2.4min finished


<h3>model evaluation</h3>

loading the model

In [44]:
filename = 'saved_model.sav'

saved_clf=pickle.load(open(filename, 'rb'))

In [45]:
saved_clf.score(X_test, y_test)

0.89608

accuracy level of almost 90 percent is not that bad